In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

scale=1.25

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=0.9*scale*20)
plt.rc("xtick", labelsize=scale*16, top=True, direction="in")
plt.rc("ytick", labelsize=scale*16, right=True, direction="in")
plt.rc("axes", titlesize=scale*20)
plt.rc("legend", fontsize=14)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
    sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    for f in font_files:
        font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

In [ ]:
for name, Qupp, Qlow, bins, ann, Eupp, cupp, cstep in zip(("22al_28h", "26p_24h"), (8.0, 10.3), (0.6, 0.6), 
                                                   (500, 810), ("$\mathrm{^{22}Al}$", "$\mathrm{^{26}P}$"), 
                                                   (6.5, 8.5), (70, 201), (20, 50)):
    f = uproot.open(proot+"/data/2023-07_analyses/q2p/%s/%s.root" % (name, name))
    t = f["a"]

    q2p = t.arrays(["Q2p"], "E > 50").Q2p[:,0]/1e3
    a = t.arrays(["Q2p", "E"], "E > 50")
    Q2p = ak.flatten(a.Q2p/1e3)
    E   = ak.flatten(a.E/1e3)
    dQ = 1e3*(Qupp - Qlow)/bins

    fig, axes = plt.subplots(1, 2, figsize=(3*6.4/2, 4.8), width_ratios=[2, 1])
    plt.subplots_adjust(wspace=0.01)
    plt.sca(axes[0])
    plt.plot(E, Q2p, 'k.', ms=0.8)
    plt.xlabel("$E_{i}$ (MeV)")
    plt.ylabel("$Q_{2 \mathrm{p} }$ (MeV)")
    plt.xticks(np.arange(0, 11, 1))
    plt.xticks(np.arange(0, 11, 0.5), minor=True)
    plt.yticks(np.arange(0, 11, 1))
    plt.yticks(np.arange(0, 11, 0.5), minor=True)
    plt.xlim(0, Eupp)
    plt.ylim(0, Qupp)
    plt.ylim(Qlow, Qupp)
    plt.sca(axes[1])
    plt.hist(q2p, bins=np.linspace(Qlow, Qupp, bins), color='k', lw=0.5, ls='-', histtype='step', 
             orientation='horizontal')
    plt.xlabel('Counts / %.1f keV' % dQ)
    plt.xticks(np.arange(cstep, cupp, cstep))
    plt.ylabel("")
    axes[1].yaxis.tick_right()
    axes[1].yaxis.set_ticks_position('both')
    axes[1].yaxis.set_label_position("right")
    plt.yticks(np.arange(0, 11, 1))
    plt.yticks(np.arange(0, 11, 0.5), minor=True)
    plt.ylim(Qlow, Qupp)
    #plt.text(0.8, 0.8, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
    plt.savefig('%s.png' % name, dpi=300, bbox_inches='tight')
    #plt.text(0.5, 0.3, ann, fontsize=18, ha='center', va='center', transform=plt.gca().transAxes)
    #plt.savefig('%s_Q2p_E.png' % name, dpi=300, bbox_inches='tight')
    plt.show()
    
    f.close()